In [ ]:
!pip install faker
import pandas as pd
import sqlite3
from faker import Faker
import string
import datetime

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 12.4 MB/s 


In [ ]:
try:
  conn = sqlite3.connect('sales.db')
except:
  print('ERROR en conexión')
else:
  print('Connected')

Connected


In [ ]:
faker = Faker()

In [ ]:
letters_digits = string.ascii_letters + string.digits

In [ ]:
def calculo_edad(prob: list, n: int) -> list:
  return [np.random.choice([np.random.randint(18,30), np.random.randint(30, 60), np.random.randint(60,85)], p=prob) for i in range(n)]

In [ ]:
def nombres(n: int) -> list:
  return [faker.name() for i in range(n)]

In [ ]:
def calculo_fecha(n: int):
  return [str(np.random.choice([faker.date_between_dates(datetime.date(2021,6,1), datetime.date(2021,8,31)),
                            faker.date_between_dates(datetime.date(2021,9,1), datetime.date(2021,9,30)),
                            faker.date_between_dates(datetime.date(2021,10,1), datetime.date(2021,11,28)),
                            faker.date_between_dates(datetime.date(2021,11,29), datetime.date(2021,12,28)),
                            faker.date_between_dates(datetime.date(2021,12,29), datetime.date(2022,1,31))],
                           p=[0.1,0.25,0.1,0.5,0.05])) for i in range(n)]

## Información cliente

In [ ]:
clientes = nombres(50)
ciudad = [faker.city() for i in range(10)]
clientes_ciudad = {name:np.random.choice(ciudad) for name in clientes}
edad = calculo_edad(prob=[0.15,0.55,0.3], n=50)
id_cliente = np.arange(1, 51, 1)

In [ ]:
df_clientes = pd.DataFrame({'nombre': clientes_ciudad.keys(),
              'ciudad': clientes_ciudad.values(),
              'edad': edad,
              'id_cliente': id_cliente})

## Información productos

In [ ]:
nombres_productos = ['celular','computador','tablet']
marcas = ['apple','samsung','huawei']
dic_price = {'celular':3000,
             'computador': 4500,
             'tablet': 2800}
dic_mult = {'apple':2.3,
            'samsung':1.8,
            'huawei':1.2}
id_producto = np.arange(1, 10, 1)

In [ ]:
prods = []
contador = 1
for nombre in nombres_productos:
  for marca in marcas:
    costo = dic_price[nombre] * dic_mult[marca]
    precio = (np.random.rand() + 1) * costo
    prods.append((nombre, marca, round(costo), round(precio), contador))
    contador += 1

In [ ]:
df_productos = pd.DataFrame(prods, columns=['nombre','marca','costo','precio', 'id_producto'])

## Información tienda

In [ ]:
ciudad_tienda = np.random.choice(ciudad, replace=False, size=3)
direccion_tienda = [faker.address() for i in range(len(ciudad_tienda))]
id_tienda = np.arange(1, 4, 1)

In [ ]:
df_tienda = pd.DataFrame({'ciudad':ciudad_tienda,
              'direccion': direccion_tienda,
              'id_tienda': id_tienda})

## Información managers

In [ ]:
nombre_manager = nombres(3)
edad_manager = calculo_edad(prob=[0.01,0.89,0.1], n=3)
id_tienda_manager = np.arange(1, 4, 1)
id_manager = np.arange(1, 4, 1)

In [ ]:
df_manager = pd.DataFrame({'nombre': nombre_manager,
                          'edad': edad_manager,
                          'id_tienda': id_tienda_manager,
                          'id_manager': id_manager})

## Información vendedores

In [ ]:
nombre_vendedores = nombres(15)
edad_vendedores = calculo_edad(prob=[0.94,0.05,0.01], n=15)
id_tienda_vendedor = np.random.choice(id_tienda, size=15)
id_vendedor = np.arange(1, 16, 1)

In [ ]:
df_vendedores = pd.DataFrame({'nombre': nombre_vendedores,
              'edad':edad_vendedores,
              'id_tienda':id_tienda_vendedor,
              'id_vendedor':id_vendedor})

## Información ventas

In [ ]:
id_venta = [''.join(np.random.choice(list(letters_digits), size=30)) for i in range(15000)]
id_vendedor_venta = np.random.choice(id_vendedor, size=15000)
id_cliente_venta = np.random.choice(id_cliente, size=15000)
id_producto_venta = np.random.choice(id_producto, size=15000)
fecha = calculo_fecha(15000)
forma_pago = np.random.choice(['efectivo','debito','credito'], size=15000)
cantidad = np.random.choice([1,2,3,4], p=[0.55,0.4,0.04,0.01], size=15000)

In [ ]:
df_ventas = pd.DataFrame({'id_venta': id_venta,
              'id_vendedor': id_vendedor_venta,
              'id_cliente': id_cliente_venta,
              'id_producto': id_producto_venta,
              'fecha': fecha,
              'forma_pago': forma_pago,
              'cantidad': cantidad}).sort_values(by='fecha').reset_index(drop=True)

# Carga de datos a base de datos

## clientes

In [ ]:
#crear tabla

query = """
CREATE TABLE clientes (
  nombre VARCHAR(150),
  ciudad VARCHAR(100),
  edad INT,
  id_cliente INT
)
"""

cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

#incluir datos en tabla

#commit

In [ ]:
for _, row in df_clientes.iterrows():
  query = f"""INSERT INTO clientes (nombre, ciudad, edad, id_cliente)
  VALUES ('{row['nombre']}', '{row['ciudad']}', {row['edad']}, {row['id_cliente']})
  """
  cursor.execute(query)
  conn.commit()

## productos

In [ ]:
query = """
CREATE TABLE productos (
  nombre VARCHAR(50),
  marca VARCHAR(50),
  costo INT,
  precio INT,
  id_producto INT
)
"""


cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

In [ ]:
for _, row in df_productos.iterrows():
  query = f"""INSERT INTO productos (nombre, marca, costo, precio, id_producto)
  VALUES ('{row['nombre']}', '{row['marca']}', {row['costo']}, {row['precio']}, {row['id_producto']})
  """
  cursor.execute(query)
  conn.commit()

## tienda

In [ ]:
query = """
CREATE TABLE tiendas (
  ciudad VARCHAR(150),
  direccion VARCHAR(150),
  id_tienda INT
)
"""


cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

In [ ]:
for _, row in df_tienda.iterrows():
  query = f"""INSERT INTO tiendas (ciudad, direccion, id_tienda)
  VALUES ('{row['ciudad']}', '{row['direccion']}', {row['id_tienda']})
  """
  cursor.execute(query)
  conn.commit()

## managers

In [ ]:
query = """
CREATE TABLE managers (
  nombre VARCHAR(50),
  edad INT,
  id_tienda INT,
  id_manager INT
)
"""


cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

In [ ]:
for _, row in df_manager.iterrows():
  query = f"""INSERT INTO managers (nombre, edad, id_tienda, id_manager)
  VALUES ('{row['nombre']}', {row['edad']}, {row['id_tienda']}, {row['id_manager']})
  """
  cursor.execute(query)
  conn.commit()

## vendedores

In [ ]:
query = """
CREATE TABLE vendedores (
  nombre VARCHAR(50),
  edad INT,
  id_tienda INT,
  id_vendedor INT
)
"""


cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

In [ ]:
for _, row in df_vendedores.iterrows():
  query = f"""INSERT INTO vendedores (nombre, edad, id_tienda, id_vendedor)
  VALUES ('{row['nombre']}', {row['edad']}, {row['id_tienda']}, {row['id_vendedor']})
  """
  cursor.execute(query)
  conn.commit()

## ventas

In [ ]:
query = """
CREATE TABLE ventas (
  id_venta VARCHAR(120),
  id_vendedor INT,
  id_cliente INT,
  id_producto INT,
  fecha VARCHAR(50),
  forma_pago VARCHAR(60),
  cantidad INT
)
"""


cursor = conn.cursor()
cursor.execute(query)
#commit

conn.commit()

In [ ]:
for _, row in df_ventas.iterrows():
  query = f"""INSERT INTO ventas (id_venta, id_vendedor, id_cliente, id_producto, fecha, forma_pago, cantidad)
  VALUES ('{row['id_venta']}', {row['id_vendedor']}, {row['id_cliente']}, {row['id_producto']}, '{row['fecha']}','{row['forma_pago']}',{row['cantidad']})
  """
  cursor.execute(query)
  conn.commit()

## Escribir queries

In [ ]:
def query():
  q = input("Escriba aquí la query: ")
  return pd.read_sql(q, con=conn)

## Cómo escribir queries

En esta sección solo debe llamarse a la función, ejecutarla y a continuación se les presentará un espacio donde pueden escribir. Escriban cualquier query haciendo uso del lenguaje SQL.

In [ ]:
query()

Escriba aquí la query: SELECT * FROM ventas


,id_venta,id_vendedor,id_cliente,id_producto,fecha,forma_pago,cantidad
0,5OerulUwGlrlEAyFNLN4DB04yxlHhK,14,17,2,2021-06-01,debito,2
1,G3zM2P0AsvVWsB2KJ6DIcy8yA025a4,15,36,7,2021-06-01,efectivo,1
2,8CeLBZQmCskdC82cxNAcGJzn8kya7P,6,24,7,2021-06-01,efectivo,3
3,kC9jguu2d3HT2a4YsDgHIKXPvqaOf3,11,14,8,2021-06-01,debito,2
4,efAmYQWtLx0r427piumgoBHLJRiX1s,6,39,4,2021-06-01,efectivo,1
...,...,...,...,...,...,...,...
14995,EjVcyMzTf6GdONHv6BoTrTYvvBkisu,10,15,6,2022-01-30,debito,1
14996,oGKRZKPX1i2602TDOv5gySROVAnVIc,3,2,5,2022-01-30,debito,2
14997,g0UyyzaSKnQObFHc4fzAN86n0mxGEo,15,48,8,2022-01-30,debito,2
14998,zoOYFDaRw8S7KJaSHaXwalj72vdWNU,4,42,1,2022-01-30,credito,2
